In [ ]:
#do not run unless new environment is created
#%pip install torch==1.13.0
#%pip install opencv-python
#%pip install torchvision==0.14.0
#%pip install opencv-python

In [1]:
import torch
import torchvision
print('torch version: ', torch.__version__)
print('torchvision version: ', torchvision.__version__)

import cv2
print('cv2 version: ', cv2.__version__)


torch version:  1.13.0
torchvision version:  0.14.0
cv2 version:  4.8.0


In [2]:
#print torch version
print(torch.__version__)
#print opencv version
print(cv2.__version__)
#print torchvision version  
print(torchvision.__version__)

1.13.0
4.8.0
0.14.0


In [3]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from os import path
import os

In [4]:
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.transforms import functional as F

In [5]:
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

/Users/david/anaconda3/envs/python3_9dis/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/david/anaconda3/envs/python3_9dis/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [6]:
#   PROCEDURE:  IMPORT_PIC(PATH)    OUTPUT: IMG, COPY

def import_pic(path):
    img = cv2.imread(path)
    copy = img.copy()
    return img, copy

In [7]:
#   PROCEDURE:  TO_TENSOR(IMG)  OUTPUT: IMAGE_TENSOR

def to_tensor(img):
    image_tensor = F.to_tensor(img)
    return image_tensor

In [8]:
#   PROCEDURE:  MAKE_PREDICTION(IMAGE_TENSOR, MODEL)    OUTPUT: PREDICTION

def make_prediction(image_tensor, model):
    prediction = model([image_tensor])
    return prediction

In [9]:
#   PROCEDURE:  BOUNDING_OBJECTS(PREDICTION)    OUTPUT: BOXES, LABELS, MASKS

def bounding_objects(prediction):
    boxes = prediction[0]['boxes']
    labels = prediction[0]['labels']
    masks = prediction[0]['masks']
    return boxes, labels, masks

In [10]:
#   PROCEDURE:  APPLY(IMAGE, BOXES, LABELS, MASKS)  OUTPUT: IMAGE

def apply(image, boxes, labels, masks):
    for i in range(len(boxes)):
        if labels[i] == 18:
            box = boxes[i].detach().numpy()
            mask = masks[i, 0].detach().numpy()
            mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

            mask = mask.astype(image.dtype)

            image = cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
            image = cv2.addWeighted(image, 0.5, mask, 0.5, 0)

#    plt.figure(figsize=(20, 20))
#    plt.imshow(image)
#    plt.axis('off')
#    plt.show()
    return image


In [11]:
#   PROCEDURE: FIND_LARGEST_BOX(BOXES)  OUTPUT: LARGEST_BOX_COORD

def find_largest_box(boxes):
    largest_box = 0
    for i in range(len(boxes)):
        box = boxes[i].detach().numpy()
        if (box[2]-box[0])*(box[3]-box[1]) > largest_box:
            largest_box = (box[2]-box[0])*(box[3]-box[1])
            largest_box_coord = box

    return largest_box_coord

In [12]:
#   PROCEDURE: CROP(IMG, BOX_COORD, SIZE) OUTPUT: CROP_IMG

def crop(img, box_coord, size):
    v_size = box_coord[3] - box_coord[1]
    h_size = box_coord[2] - box_coord[0]

    assert v_size < size, ("dog object larger than cropping size vertically", v_size, ' > ', size)
    assert h_size < size,("dog object is larger than cropping size horizontally", h_size , ' > ' , size)

    v_diff = size - v_size
    h_diff = size - h_size

    half_vdiff = v_diff/2
    half_hdiff = h_diff/2

    new_coord0 = box_coord[0] - half_hdiff
    new_coord1 = box_coord[1] - half_vdiff
    new_coord2 = box_coord[2] + half_hdiff
    new_coord3 = box_coord[3] + half_vdiff
    
    if new_coord0 < 0:
        new_coord0 = 0
        new_coord2 = size

    if new_coord1 < 0:
        new_coord1 = 0
        new_coord3 = size

    if new_coord2 > img.shape[1]:
        new_coord0 = img.shape[1] - size
        new_coord2 = img.shape[1]

    if new_coord3 > img.shape[0]:
        new_coord1 = img.shape[0] - size
        new_coord3 = img.shape[0]

    crop_img = img[int(new_coord1) : int(new_coord3),int(new_coord0) : int(new_coord2)]
    return crop_img

    



In [13]:
#   PROCEDURE: EXPORT(IMG, NAME, EP)    OUTPUT: BOOLEAN

def export(img, name, ep):
    if not os.path.exists(ep):
        os.makedirs(ep)

    path = ep + '/' + name + '.jpg'
    cv2.imwrite(path, img)
    #check if image was sucessfully written
    if os.path.isfile(path):
        
        return True
    else:
        
        return False
        

In [14]:
#  PROCEDURE: PIPELINE(MODEL, IP, NAME, EP, SIZE)    OUTPUT: BOOLEAN

def pipeline(model, ip, name, ep, size=500):
    img, copy = import_pic(ip)
    image_tensor = to_tensor(img)
    prediction = make_prediction(image_tensor, model)
    boxes, labels, masks = bounding_objects(prediction)
    applied_img = apply(copy, boxes, labels, masks)
    largest = find_largest_box(boxes)
    cropped = crop(img, largest, size)
    exported = export(cropped, name, ep)

    
    #plt.imshow(cropped)
    #plt.axis('off')
    #plt.show()

    if exported == True:
        print('Cropped image successfully exported as :', name, '.jpg \n ~~to the folder: ', ep)
        return True
    else:
        print('Export failed, please try again')


In [15]:
#  PROCEDURE: ITER_PIP(MODEL, IP, EP, END, START, COUNTER_START)    OUTPUT: NONE

def iter_pip(model, ip, ep, end, start= 0, counter_start=0):
    i = start
    counter = counter_start
    
    while i < end:
        try:
            img_path = ip + '/' + str(i) + '.jpg'
            pipeline(model, img_path, str(counter), ep)
        
        except AssertionError as e:
            print(e)
            print('Image skipped')
            pass
        counter += 1
        i += 1

In [16]:
iter_pip(model,
        '/Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/frames/laying', 
        '/Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying', 
        156)

Cropped image successfully exported as : 0 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying
Cropped image successfully exported as : 1 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying
Cropped image successfully exported as : 2 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying
Cropped image successfully exported as : 3 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying
Cropped image successfully exported as : 4 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying
Cropped image successfully exported as : 5 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying
Cropped image successfully exported as : 6 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/laying

In [20]:
iter_pip(model,
        '/Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/frames/standing',
        '/Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/standing',
        122)

Cropped image successfully exported as : 0 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/standing
Cropped image successfully exported as : 1 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/standing
Cropped image successfully exported as : 2 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/standing
Cropped image successfully exported as : 3 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/standing
Cropped image successfully exported as : 4 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/standing
Cropped image successfully exported as : 5 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/standing
Cropped image successfully exported as : 6 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cr

In [21]:
iter_pip(model,
        '/Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/frames/sitting',
        '/Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/sitting',
        146)

Cropped image successfully exported as : 0 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/sitting
Cropped image successfully exported as : 1 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/sitting
Cropped image successfully exported as : 2 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/sitting
Cropped image successfully exported as : 3 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/sitting
Cropped image successfully exported as : 4 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/sitting
Cropped image successfully exported as : 5 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/sitting
Cropped image successfully exported as : 6 .jpg 
 ~~to the folder:  /Users/david/Desktop/shelter_dogs_research/Data/trial1_pup1/cropped/